In [1]:
import sys, os
import time, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as cnt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns


plt.style.use(style='default')
#print(plt.style.available)
#bmh, classic, default, fivethirtyeight, ggplot ...
plt.style.use(style='bmh')


from bs4 import BeautifulSoup
from selenium import webdriver # naršyklės kontroleris
from selenium.webdriver.chrome.options import Options #Naršyklės  parametrai


import sqlite3
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
DB = sqlite3.connect('Baigiamasis_Darbas_varle.db')
C = DB.cursor()

In [3]:
print(os.getcwd())

e:\Mokslai


In [4]:
SQL = '''create table if not exists Telefonai7
(
    id integer primary key autoincrement,
    Gamintojas text,
    Pavadinimas text,
    Spalva text,
    Vidinė_atmintis_GB integer,
    Procesoriaus_tipas text,
    Ekrano_raiška integer,
    Atmintis_Ram integer,
    Ekrano_įstrižainė_coliais integer,
    Operacinė_sistema text,
    Kaina interger,
    Kaina_Men integer,
    Nuolaida integer,
    Likutis integer,
    Ivertinimai integer,
    Atvezimas integer

)
'''

C.execute(SQL)
DB.commit()

In [5]:
opcijos = Options()
driver = uc.Chrome(options=opcijos)

all_data = []

wait = WebDriverWait(driver, 1)

def spec_value(sk, label):
    li = next((x for x in sk.select(".spec-shortcuts li")
               if label in x.get_text(" ", strip=True)), None)
    if not li:
        return "Nera parasyta"
    v = li.select_one("a span, button.redirect-btn span")
    return v.get_text(strip=True) if v else "Nera parasyta"


page = 1
while True:
    driver.get(f"https://www.varle.lt/mobilieji-telefonai/?p={page}")

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.ajax-container")))
    except TimeoutException:
        break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    skelbimai = soup.select("div.GRID_ITEM")
    

    if not skelbimai:
        break




    for sk in skelbimai:
        
        
        

        g = sk.select_one('.spec-shortcuts a[href*="f.s_gamintojas"] span')
        Pavadinimas_el = sk.select_one(".product-title")
        s = sk.select_one('.spec-shortcuts a[href*="f.s_spalva"] span')
        o = sk.select_one(' .spec-shortcuts a [href*="f.s-operacine_sistema"] span')
        Kaina_el = sk.select_one(".price-value")
        Kaina_Men_el = sk.select_one(".top-line")
        Nuolaida_el = sk.select_one(".discount-line")
        Likutis_el = sk.select_one(".stock")
        Ivertinimai_el = sk.select_one(".total-ratings")
        Atvezimas_el = sk.select_one(".delivery")

        gamintojas = g.get_text(strip=True) if g else next((x.select_one("a span").get_text(strip=True)
            for x in sk.select(".spec-shortcuts li") if "Gamintojas" in x.get_text(" ", strip=True) and x.select_one("a span")), "Nera parasyta")
        
        Pavadinimas = Pavadinimas_el.get_text(strip=True) if Pavadinimas_el else None
        Spalva = s.get_text(strip=True) if s else next ((b.select_one("a span").get_text(strip=True)
            for b in sk.select(".spec-shortcuts li") if "Spalva" in b.get_text( " ", strip=True) and b.select_one("a span")), "Nera parasyta")
        
        
        Ekrano_istrizaine   = spec_value(sk, "Ekrano įstrižainė")
        Vidinė_atmintis     = spec_value(sk, "Vidinė atmintis")
        Atmintis_ram        = spec_value(sk, "Atmintis (RAM)")
        Procesoriaus_tipas  = spec_value(sk, "Procesoriaus tipas")
        Ekrano_raiska       = spec_value(sk, "Ekrano raiška")

        
        
        Operacine = o.get_text(strip=True) if o else next ((u.select_one("a span").get_text(strip=True)
            for u in sk.select(".spec-shortcuts li") if "operacin" in u.get_text(" ", strip=True).lower() and u.select_one("a span")) , "Nera parasyta")
        Kaina = Kaina_el.get_text(strip=True) if Kaina_el else "Nera parasyta"
        Kaina_Men = Kaina_Men_el.get_text(strip=True) if Kaina_Men_el else "Nera parasyta"
        Nuolaida = Nuolaida_el.get_text(strip=True) if Nuolaida_el else "Nera parasyta"
        Likutis = Likutis_el.get_text(strip=True) if Likutis_el else "Nera parasyta"
        Ivertinimai = Ivertinimai_el.get_text(strip=True) if Ivertinimai_el else "Nera parasyta"
        
        Atvezimas = Atvezimas_el.get_text(strip=True) if Atvezimas_el else "Nera parasyta"


        all_data.append((gamintojas, Pavadinimas, Spalva, Vidinė_atmintis, Procesoriaus_tipas, Ekrano_raiska, Atmintis_ram, Ekrano_istrizaine,
                     Operacine, Kaina, Kaina_Men, Nuolaida, Likutis, Ivertinimai, Atvezimas))

    page += 1

driver.quit()


SQLinsert = '''insert into Telefonai7
    (Gamintojas, Pavadinimas, Spalva, Vidinė_atmintis_GB ,Procesoriaus_tipas, Ekrano_raiška, Atmintis_Ram, Ekrano_įstrižainė_coliais,
    Operacinė_sistema, Kaina, Kaina_Men, Nuolaida, Likutis, Ivertinimai, Atvezimas)
    values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''




C.executemany(SQLinsert, all_data)
DB.commit()

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=145.0.7632.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x70f3e3
	0x70f424
	0x502090
	0x4f0fee
	0x50fb03
	0x5756cc
	0x58b4b9
	0x56ebb6
	0x540319
	0x5410d4
	0x9646a4
	0x95f9a9
	0x97c940
	0x7280f8
	0x72fe4d
	0x717c18
	0x717de2
	0x7018ea
	0x75a95d49
	0x773dd83b
	0x773dd7c1
